In [2]:
import bs4 as bs
import requests

In [1]:
def get_soup(url):
    response = requests.get(url)
    soup = bs.BeautifulSoup(response.text, 'html.parser')
    return soup

In [17]:
def get_content_course(url: str) -> str:
    soup = get_soup(url)
    container  = soup.find('div', class_="luna-content_luna-content__section__z7rg5")
    if not container:
        return ""
    else:
        return container.get_text(separator=' ', strip=True)

In [63]:

def get_courses_from_html(url: str) -> dict:
    soup = get_soup(url)
    container = soup.find("div", class_="program-template_program__content__w3oIs")
    chapters = {}
    if not container:
        return chapters

    chapter_divs = container.find_all("div", class_="course-list_chapter__13m89")
    courses_list = []
    for chapter in chapter_divs:
        # Récupérer le titre du chapitre depuis le h2
        header = chapter.find("div", class_="course-list_chapter__title--without-cups__dehJf")
        h2 = header.find("h2") if header else None
        chapter_title = h2.get_text(strip=True) if h2 else "Sans titre"

        
        ul = chapter.find("ul", class_="course-list_chapter__course-list__x_5Q6")
        if ul:
            for li in ul.find_all("li", class_="course-list_chapter__course-list--item-without-cups__CJUPl"):
                a = li.find("a", class_="course-title-card-module_course-title-card__YrMOq")
                if a:
                    link = a.get("href")
                    p = a.find("p")
                    course_title = p.get_text(strip=True) if p else ""
                    courses_list.append({
                        "link": link,
                        "title": course_title,
                        "content": get_content_course(link),
                        "theme": chapter_title
                    })
          
       
            
            
        

    return courses_list

In [49]:
url_prog_school= "https://www.schoolmouv.fr/3eme/histoire"

In [64]:
courses = get_courses_from_html(url_prog_school)

In [65]:
courses

[{'link': 'https://www.schoolmouv.fr/cours/la-premiere-guerre-mondiale-vers-une-guerre-totale/fiche-de-cours',
  'title': 'La Première Guerre mondiale : vers une guerre totale',
  'content': 'Introduction\xa0: Pour les historiens, le monde entre dans le XX e siècle à partir de la Première Guerre mondiale. En effet, celle-ci représente un tournant majeur dans l’histoire de l’humanité, de par sa violence et ses conséquences. S’il est important d’en connaître la chronologie, il s’agit surtout de comprendre son importance dans l’histoire du XX e siècle. En quoi la Première Guerre mondiale peut être qualifiée de guerre totale\xa0? Pourquoi ce conflit inaugure-t-il une période de violence caractéristique du XX e siècle\xa0? Pour répondre à ces questions, nous analyserons tout d’abord les grandes phases de la guerre, puis nous étudierons les raisons pour lesquelles on peut parler d’une guerre nouvelle, à partir de l’exemple de la bataille de Verdun. Finalement nous analyserons les conséquence

In [20]:
key = list(courses.keys())[0]
print(courses[key]["content"])

​Introduction : En 1939, la France et l’Angleterre déclarent la guerre à l’Allemagne, après que celle-ci ait envahi la Pologne. Mais en moins d’un an, la France est battue par l’armée allemande. Les conséquences politiques et sociales de cette défaite sont nombreuses et complexes. La III e République, qui avait été fondée en 1870 et qui avait survécu à la Première Guerre disparaît ; un nouveau régime politique se met en place : le régime de Vichy, avec à sa tête l’ex-maréchal Pétain, vainqueur de la bataille de Verdun pendant la Première Guerre. La Résistance à Pétain et au nazisme se met alors en place depuis l’étranger et à l’intérieur du pays : des réseaux de résistants organisent le futur de la France et participent  à la libération du pays en 1944, avec l’aide des troupes anglo-américaines. Dans ce cours, nous allons voir tout d’abord comment la France a perdu si rapidement la guerre et comment s’installe le régime de Vichy. Nous étudierons ensuite les relations entre le régime de

In [39]:
import pandas as pd
df = pd.DataFrame(courses)

In [40]:
df

,"L'Europe, un théâtre majeur des guerres totales (1914-1945)",Le monde depuis 1945,Françaises et Français dans une République repensée
link,https://www.schoolmouv.fr/cours/la-france-de-v...,https://www.schoolmouv.fr/cours/le-nouvel-ordr...,https://www.schoolmouv.fr/cours/les-trente-glo...
title,La France de Vichy (1940-1944),La géopolitique du monde actuel,Les Trente Glorieuses
content,"​Introduction : En 1939, la France et l’Anglet...",Introduction : L’effondrement du bloc soviétiq...,Introduction : L’expression « Trente Glorieuse...


In [2]:
import sqlite3

def create_db(db_path: str = "courses.db"):
    """
    Crée la base de données et la table course_info si elles n'existent pas.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS course_info (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            site TEXT,
            matiere TEXT,
            theme TEXT,
            chapitre TEXT,
            content TEXT,
            link TEXT
        )
    """)
    conn.commit()
    conn.close()

def insert_course(db_path: str, site: str, matiere: str, theme: str, chapitre: str, content: str, link: str):
    """
    Insère une nouvelle entrée dans la table course_info.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO course_info (site, matiere, theme, chapitre, content, link)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (site, matiere, theme, chapitre, content, link))
    conn.commit()
    conn.close()


In [5]:
db_file = "courses.db"
create_db(db_file)

In [71]:
# Insertion des données dans la base de données
for course in courses:
    insert_course(db_file, "schoolmouv", "Histoire", course["theme"], course["title"], course["content"], course["link"])


In [10]:
# Vérifier l'insertion
conn = sqlite3.connect("db/courses.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM course_info")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

(1, 'schoolmouv', 'Histoire', "L'Europe, un théâtre majeur des guerres totales (1914-1945)", 'La Première Guerre mondiale : vers une guerre totale', 'Introduction\xa0: Pour les historiens, le monde entre dans le XX e siècle à partir de la Première Guerre mondiale. En effet, celle-ci représente un tournant majeur dans l’histoire de l’humanité, de par sa violence et ses conséquences. S’il est important d’en connaître la chronologie, il s’agit surtout de comprendre son importance dans l’histoire du XX e siècle. En quoi la Première Guerre mondiale peut être qualifiée de guerre totale\xa0? Pourquoi ce conflit inaugure-t-il une période de violence caractéristique du XX e siècle\xa0? Pour répondre à ces questions, nous analyserons tout d’abord les grandes phases de la guerre, puis nous étudierons les raisons pour lesquelles on peut parler d’une guerre nouvelle, à partir de l’exemple de la bataille de Verdun. Finalement nous analyserons les conséquences de la guerre. Les grandes phases de la g

In [49]:
db_file = "db/quiz.db"
con = sqlite3.connect(db_file)
cursor = con.cursor()
cursor.execute("SELECT * FROM questions")
rows = cursor.fetchall()
for row in rows:
    print(row)
con.close()


(1, "Quel principe fondamental de la République française est le plus directement lié à l'idée de redéfinir et de renforcer la démocratie dans le contexte actuel ?", 'La laïcité', "L'égalité", 'La fraternité', 'La liberté', 1, 'Histoire', 'Françaises et Français dans une République repensée', "Pensez à l'importance de donner à chaque citoyen les mêmes chances et les mêmes droits dans une société démocratique.", "La redéfinition et le renforcement de la démocratie nécessitent de garantir que tous les citoyens ont des droits égaux et une voix égale dans les processus décisionnels. L'égalité est donc le principe le plus directement lié à cette idée, car elle assure que chaque individu a les mêmes opportunités et les mêmes droits, ce qui est essentiel pour une démocratie fonctionnelle et inclusive.")
(2, 'Quel concept philosophique, issu des Lumières, a le plus influencé la conception moderne de la citoyenneté dans la République française, notamment en ce qui concerne la redéfinition des d

In [70]:
# affiche toutes les tables de la base de données
db_file = "db/quiz.db"
con = sqlite3.connect(db_file)
cursor = con.cursor()
cursor.execute("SELECT * from quizzes")
rows = cursor.fetchall()
for row in rows:
    print(row)
    




(1, 'Histoire', "L'Europe, un théâtre majeur des guerres totales (1914-1945)", '2025-02-02 12:06:46')


In [78]:
# affiche toutes les tables de la base de données
db_file = "db/quiz.db"
con = sqlite3.connect(db_file)
cursor = con.cursor()
cursor.execute("SELECT * from answers")
rows = cursor.fetchall()
for row in rows:
    print(row)
    



(1, 1, 3, 21, 4, 0, 5.0, 0)
(2, 1, 3, 21, 4, 0, 5.0, 0)
(3, 1, 3, 22, 2, 0, 5.0, 0)
(4, 1, 3, 23, 2, 0, 5.0, 0)
(5, 1, 3, 24, 1, 0, 5.0, 0)
(6, 1, 3, 25, 1, 1, 5.0, 0)
(7, 1, 3, 26, 1, 0, 5.0, 0)
(8, 1, 3, 27, 3, 0, 5.0, 0)
(9, 1, 3, 28, 2, 0, 5.0, 0)
(10, 1, 3, 29, 1, 0, 5.0, 0)
(11, 1, 3, 30, 1, 0, 5.0, 0)
(12, 1, 3, 21, 4, 0, 39.86342930793762, 1)
(13, 1, 3, 22, 1, 1, 6.0753514766693115, 0)
(14, 1, 3, 23, 1, 0, 5.351883172988892, 0)
(15, 1, 3, 21, 3, 0, 6.098170280456543, 0)
(16, 1, 3, 22, 2, 1, 4.07360315322876, 0)
(17, 1, 3, 23, 1, 1, 6.544925212860107, 0)
(18, 1, 3, 24, 3, 1, 1.026299238204956, 0)
(19, 1, 3, 25, 2, 1, 1.127025842666626, 0)
(20, 1, 3, 26, 4, 0, 1.0276076793670654, 0)
(21, 1, 3, 27, 4, 0, 1.0203700065612793, 0)
(22, 1, 3, 28, 4, 0, 1.2228424549102783, 0)
(23, 1, 3, 29, 2, 0, 1.087273120880127, 0)
(24, 1, 3, 30, 1, 0, 0.8893251419067383, 0)


In [6]:
db_file = "db/rag_metrics.db"
con = sqlite3.connect(db_file)
cursor = con.cursor()
cursor.execute("SELECT * FROM rag_metrics")
rows = cursor.fetchall()
for row in rows:
    print(row)
con.close()


(1, '2025-02-01T22:01:24.131305', 172, 40, 0.00033024, 0.00023, 0.0, 0.00029981291200355785, 0.0034619122291480904)
(2, '2025-02-01T22:02:06.165310', 172, 66, 0.00033024, 0.0003795, 0.0, 0.0005066929313076639, 0.005787087729161574)
(3, '2025-02-01T22:30:28.027115', 162, 65, 0.00031104, 0.00037374999999999995, 1.756425380706787, 0.0004853109619584403, 0.005613838187502326)
(4, '2025-02-01T22:31:30.060963', 162, 64, 0.00031104, 0.000368, 9.786977291107178, 0.0006991177372635414, 0.006908997333333334)
(5, '2025-02-01T22:33:01.738110', 162, 64, 0.00031104, 0.000368, 1.5505855083465576, 0.0004719525347334332, 0.005490683895838821)
(6, '2025-02-01T22:33:26.793723', 162, 64, 0.00031104, 0.000368, 1.5582282543182373, 0.000472144323180693, 0.005491881333337501)
(7, '2025-02-01T22:34:06.050879', 162, 64, 0.00031104, 0.000368, 1.5297822952270508, 0.0004712449459098967, 0.0054862660416716115)
(8, '2025-02-01T22:37:23.629136', 154, 65, 0.00029568, 0.00037374999999999995, 1.6615865230560303, 0.00048

In [65]:
db_file = "db/quiz.db"
con = sqlite3.connect(db_file)
cursor = con.cursor()

cursor.execute("SELECT * FROM quizzes ")
rows = cursor.fetchall()
for row in rows:
    print(row)
    